In [ ]:
# Cell 1: Imports and Setup
import pandas as pd
import numpy as np
from sksurv.ensemble import GradientBoostingSurvivalAnalysis
from sksurv.metrics import concordance_index_censored, cumulative_dynamic_auc
from sklearn.model_selection import KFold
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import matplotlib.pyplot as plt
import seaborn as sns

plt.rcParams["figure.dpi"] = 300


In [ ]:
# Cell 2: Load and preprocess data
df = pd.read_csv("full_clean.csv")

# Encode event column (discharge = event, death = censoring)
df["event"] = df["event"].astype(bool)

# Encode categorical variables
categorical_cols = ["region", "MOI", "hr_cat", "rr_cat", "sbp_cat", "sex"]
ordinal_cols = ["gcs"]
numeric_cols = ["age"]

# Define preprocessing
preprocessor = ColumnTransformer([
    ("cat", OneHotEncoder(drop="first"), categorical_cols),
    ("ord", "passthrough", ordinal_cols),
    ("num", "passthrough", numeric_cols),
])

X = preprocessor.fit_transform(df)
y = np.array([(e, t) for e, t in zip(df["event"], df["los"])], 
             dtype=[("event", bool), ("time", float)])


In [ ]:
# Cell 3: Define model and cross-validation
model = GradientBoostingSurvivalAnalysis(n_estimators=150, random_state=0)
kf = KFold(n_splits=5, shuffle=True, random_state=42)

region_list = df["region"].unique()
results_by_region = {r: [] for r in region_list}
auc_by_region = {r: [] for r in region_list}
eval_times_global = []
cindex_all = []


In [ ]:
# Cell 4: Run CV and record metrics
for fold, (train_idx, test_idx) in enumerate(kf.split(X)):
    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]
    model.fit(X_train, y_train)
    pred = model.predict(X_test)

    # Overall C-index
    c_all = concordance_index_censored(y_test["event"], y_test["time"], pred)[0]
    cindex_all.append(c_all)

    # Time-dependent AUC (overall)
    times = np.linspace(np.percentile(y_test["time"], 5), np.percentile(y_test["time"], 95), 50)
    eval_times_global = times  # save for plotting later
    _, aucs_overall = cumulative_dynamic_auc(y_train, y_test, pred, times)

    # Stratify by region
    region_test = df.iloc[test_idx]["region"].values
    for region in region_list:
        region_mask = region_test == region
        if np.sum(region_mask) < 5:
            continue  # too few samples
        y_test_r = y_test[region_mask]
        X_test_r = X_test[region_mask]
        pred_r = model.predict(X_test_r)
        c_r = concordance_index_censored(y_test_r["event"], y_test_r["time"], pred_r)[0]
        results_by_region[region].append(c_r)

        try:
            _, aucs_r = cumulative_dynamic_auc(y_train, y_test_r, model.predict(X_test_r), times)
            auc_by_region[region].append(aucs_r)
        except ValueError:
            continue


In [ ]:
# Cell 5: Summarize and print C-index
print(f"Overall C-index: {np.mean(cindex_all):.3f} ± {np.std(cindex_all):.3f}")
for region in region_list:
    if results_by_region[region]:
        cidx = results_by_region[region]
        print(f"{region} → C-index: {np.mean(cidx):.3f} ± {np.std(cidx):.3f}")


In [ ]:
# Cell 6: Plot Time-Dependent AUC by Region
plt.figure(figsize=(10, 6))

# Overall
_, aucs_all = cumulative_dynamic_auc(y, y, model.predict(X), eval_times_global)
plt.plot(eval_times_global, aucs_all, label="Overall", linestyle="--", color="black")

# Regions
for region in region_list:
    if region in auc_by_region and len(auc_by_region[region]) > 0:
        auc_stack = np.vstack(auc_by_region[region])
        auc_mean = np.nanmean(auc_stack, axis=0)
        plt.plot(eval_times_global, auc_mean, label=f"{region}")

plt.axhline(0.5, ls="--", color="gray")
plt.xlabel("Days since Admission")
plt.ylabel("Time-dependent AUC")
plt.title("Dynamic AUC Stratified by Region")
plt.ylim(0.3, 1.0)
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()
